<a href="https://colab.research.google.com/github/deltorobarba/machinelearning/blob/master/hhl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Harrow-Hassidim-Lloyd Algorithm (HHL)**

In [ ]:
!pip install cirq --quiet
import cirq
import sympy
from cirq.contrib.svg import SVGCircuit
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt # %matplotlib inline
print(cirq.__version__)

https://www.quantamagazine.org/new-quantum-algorithms-finally-crack-nonlinear-equations-20210105/

###### **<font color="blue">Promise, Disadvantages & Applications of HHL</font>**

The [quantum algorithm for linear systems of equations](https://en.m.wikipedia.org/wiki/Quantum_algorithm_for_linear_systems_of_equations) designed by Aram Harrow, Avinatan Hassidim, and Seth Lloyd.

> is a quantum algorithm for solving linear systems. 

> The algorithm estimates the result of a scalar measurement on the solution vector to a given linear system of equations.

* The algorithm is one of the main fundamental algorithms expected to provide a speedup over their classical counterparts, along with Shor's factoring algorithm, Grover's search algorithm, the quantum fourier transform and quantum simulation. 

* Provided the linear system is sparse and has a low condition number $\kappa_{1}$ and that the user is interested in the result of a scalar measurement on the solution vector, instead of the values of the solution vector itself, then the algorithm has a runtime of $O\left(\log (N) \kappa^{2}\right)$, where $N$ is the number of variables in the linear system. This offers an exponential speedup over the fastest classical algorithm, which runs in $O(N \kappa)$ (or $O(N \sqrt{\kappa})$ for positive semidefinite matrices).

**Promise**: 

* Solving 10,000 linear equation: a classical computer needs in best case 10,000 steps. HHL just 13.

* Unlike the classical solutions to the Deutsch-Jozsa and search problems, most of our classical methods for matrix manipulation do work in polynomial time. However, as data analysis becomes more and more powerful (and more and more demanding on today’s computers), the size of these matrices can make even polynomial time too long.

**Disadvantages:**

* solution vector is not yielded (rather it prepares a quantum state that is proportional to the solution): Actually reading out the solution vector would take O(N)time, so we can only maintain the logarithmic runtime by sampling the solution vector like ⟨x|M|x⟩, where M is a quantum-mechanical operator. Therefore, **HHL is useful mainly in applications where only samples from the solution vector are needed**. 

* Entries of matrix have to be sparse: Additionally, although HHL is exponentially faster than Conjugate Gradient in N, it is polynomially slower in s and 𝜅, so HHL is restricted to only those matrices that are sparse and have low condition numbers.

* Must satisfy robust invertibility (means that entries of matrix must all approx. of same size)

* Preparation of input vector is complicated


https://en.wikipedia.org/wiki/Quantum_algorithm_for_linear_systems_of_equations

https://en.wikipedia.org/wiki/Quantum_optimization_algorithms

https://www.quantamagazine.org/a-new-approach-to-multiplication-opens-the-door-to-better-quantum-computers-20190424/

https://www.quantamagazine.org/teenager-finds-classical-alternative-to-quantum-recommendation-algorithm-20180731/

https://medium.com/mit-6-s089-intro-to-quantum-computing/hhl-solving-linear-systems-of-equations-with-quantum-computing-efb07eb32f74

https://qiskit.org/textbook/ch-applications/hhl_tutorial.html

https://en.m.wikipedia.org/wiki/Quantum_algorithm_for_linear_systems_of_equations

<font color="black">**Applications of HHL & Familiar methods**

* Systems of linear equations arise naturally in many real-life applications in a wide range of areas, such as in the solution of Partial Differential Equations, the calibration of financial models, fluid simulation or numerical field calculation. 

* Makes use of Quantum Phase Estimation

* Quantum Matrix Inversion

* **Used in many quantum machine learning algorithms as a building block**


**Applications of HHL**

* The quantum algorithm for linear systems of equations has been applied to a support vector machine, which is an optimized linear or non-linear binary classifier (https://arxiv.org/abs/1307.0471v2)

* for Least-squares fitting (https://arxiv.org/abs/1204.5242)

* for finite-element-methods (https://arxiv.org/abs/1512.05903) (but only for higher problems which include solutions with higher-order derivatives and large spatial dimensions. For example, problems in many-body dynamics require the solution of equations containing derivatives on orders scaling with the number of bodies, and some problems in computational finance, such as Black-Scholes models, require large spatial dimensions)

**Familiar methods of solutions**

* Substitution method
* Graphical method
* Matrix method
* Cramer's rule
* Gaussian elimination

###### <font color="blue">**Solving Systems of Linear Equations: Squared Matrix (for HHL)**

<font color="red">**If A is squared is a matrix (and has [full rank](https://de.m.wikipedia.org/wiki/Rang_(Mathematik))) in a linear system of equations:**

> $A x = b$

then you can take the [inverse](https://de.m.wikipedia.org/wiki/Inverse_Matrix) if A to solve for x:

> $x = A^{-1} b$

<font color="red">*This part is important for HHL:*

* $\hat{A} = \hat{A}^{\dagger}$ $\quad$ - Hermitian operators are [Self-adjoint operators](https://en.m.wikipedia.org/wiki/Self-adjoint_operator)

* Adjungierte Matrix = transponiert + complex konjugiert (Vorzeichen umgekehrt). Hermetian: selbstadjunktiert = symmetrisch

* Following from this, in bra-ket notation: $
\left\langle\phi_{i}|\hat{A}| \phi_{j}\right\rangle=\left\langle\phi_{j}|\hat{A}| \phi_{i}\right\rangle^{*}
$


<font color="red">*Since $A$ is Hermitian, it has a spectral decomposition : $
A=\sum_{j=0}^{N-1} \lambda_{j}\left|u_{j}\right\rangle\left\langle u_{j}\right|, \quad \lambda_{j} \in \mathbb{R}
$*

<font color="red">*You need the Eigendecomposition (spectral decomposition) to get the inverse of a matrix (=here unitary and hence normal)*

Getting the [Matrix inverse via eigendecomposition](https://en.m.wikipedia.org/wiki/Eigendecomposition_of_a_matrix#Matrix_inverse_via_eigendecomposition): If a matrix $\mathbf{A}$ can be eigendecomposed and if none of its eigenvalues are zero, then $\mathbf{A}$ is invertible and its inverse is given by

>$
\mathbf{A}^{-1}=\mathbf{Q}^{-1} \mathbf{\Lambda}^{-1} \mathbf{Q}
$

If $\mathbf{A}$ is a symmetric matrix, since $\mathbf{Q}$ is formed from the eigenvectors of $\mathbf{A}, \mathbf{Q}$ is guaranteed to be an orthogonal matrix, therefore $\mathbf{Q}^{-1}=\mathbf{Q}^{\mathrm{T}}$. Furthermore, because $\mathbf{\Lambda}$ is a diagonal matrix, its inverse is easy to calculate:

>$
\left[\Lambda^{-1}\right]_{i i}=\frac{1}{\lambda_{i}}
$

*Eigendecomposition is the factorization of a matrix into a canonical form, whereby the matrix is represented in terms of its eigenvalues and eigenvectors. **Only diagonalizable matrices can be factorized in this way**. When the matrix being factorized is a [normal](https://en.m.wikipedia.org/wiki/Normal_matrix) or [real symmetric matrix](https://en.m.wikipedia.org/wiki/Symmetric_matrix), the decomposition is called "spectral decomposition", derived from the [**spectral theorem**](https://en.m.wikipedia.org/wiki/Spectral_theorem).*

  * *Among complex matrices, all unitary, Hermitian, and skew-Hermitian matrices are normal*

  * *The spectral theorem states that a matrix is normal if and only if it is unitarily similar to a diagonal matrix, and therefore any matrix A satisfying the equation $A^*A = AA^*$ is diagonalizable.*

  * *A complex square matrix A is normal if it commutes with its conjugate transpose $A^*$ : $A$ normal $\Longleftrightarrow A^{*} A=A A^{*}$*

There are also other ways to get the matrix inverse besides Eigendecomposition, for example Gaussian Elemination, Cramers rule, mewtons method, cholesky decomposition: https://en.m.wikipedia.org/wiki/Invertible_matrix

https://en.m.wikipedia.org/wiki/Eigendecomposition_of_a_matrix#Matrix_inverse_via_eigendecomposition

###### <font color="blue">**Solving Systems of Linear Equations: Non-Squared Matrix (Linear Least Squares)**

*In linear algebra, the singular value decomposition (SVD) is a factorization of a real or complex matrix. It generalizes the eigendecomposition of a square normal matrix with an orthonormal eigenbasis to any $m\times n$ matrix. It is related to the polar decomposition.*

* non squared matrix

* to solve you need the inverse, but normal eigendecomposition doesn work

> you need to compute mooore penrose pseudo-inverse, and here you need to apply singular value decomposition to get eigendecomposition

* SVD is a type of [Matrix decomposition](https://en.m.wikipedia.org/wiki/Matrix_decomposition), specicially one based on eigenvalue concepts. and SVD is a full rank decomposition (besides broader [Rank factorization methods](https://en.m.wikipedia.org/wiki/Rank_factorization))

* matrix decompositions in general are used to take a large matrix apart (i.e. factorizes a matrix into a lower triangular matrix L and an upper triangular matrix U) so the new matrices require fewer additions and multiplications to solve, compared with the original system $A\mathbf {x} =\mathbf {b}$

<font color="red">**If a matrix A is not squared:**

> $A x = b$ $\quad$ ($A$ is not regular)

You multiply both sides by the $A^{T}$, which is the transpose of A:

> $A^{T} A x = A^{T} b$

Then move $A^{T} A$ on right side (by taking their inverse). This is not the original x anymore, but the [least squares](https://de.m.wikipedia.org/wiki/Methode_der_kleinsten_Quadrate#Lineare_Modellfunktion) $\hat{x}$

> $\hat{x} =$ <font color="blue">$(A^{T} A)^{-1} A^{T}$</font> $b$

And that term <font color="blue">$(A^{T} A)^{-1} A^{T}$</font> is know as (Moore–Penrose) pseudo-inverse <font color="blue">$A^{+}$</font>:

> $\hat{x} =$ <font color="blue">$A^{+}$</font> $b$

And a pseudo-inverse is nothing else than our least-squares solutions. See more details under [Numerical methods for linear least squares](https://en.m.wikipedia.org/wiki/Numerical_methods_for_linear_least_squares)

<font color="red">**Example:**

> $A x = b$

> $\left[\begin{array}{cc}2 & -2 \\ -2 & 2 \\ 5 & 3\end{array}\right] x=\left[\begin{array}{c}-1 \\ 7 \\ -26\end{array}\right]$

* We can immediately see that matrix A is of rank 2 because the first two rows are multiples of each other (2 and -2 and -2 and 2), just the last row numbers are not multiples of each other (5 and 3).

* Now we can apply the pseudo-inverse to find the least-squares solution:

> $\hat{x} =$ <font color="blue">$(A^{T} A)^{-1} A^{T}$</font> $b$

> $\hat{x}=$ <font color="blue">$\left(\left[\begin{array}{ccc}2 & -2 & 5 \\ -2 & 2 & 3\end{array}\right]\left[\begin{array}{cc}2 & -2 \\ -2 & 2 \\ 5 & 3\end{array}\right]\right)^{-1}\left[\begin{array}{ccc}2 & -2 & 5 \\ -2 & 2 & 3\end{array}\right]$</font>$\left[\begin{array}{c}-1 \\ 7 \\ -26\end{array}\right]$

> $\hat{x}=$$\left[\begin{array}{l}-4 \\ -2\end{array}\right]$


*Inserting the least-squares $\hat{x}$ into the original equation:*

> $\left[\begin{array}{cc}2 & -2 \\ -2 & 2 \\ 5 & 3\end{array}\right]\left[\begin{array}{l}-4 \\ -3\end{array}\right]$ = $\left[\begin{array}{c}2 \cdot(-4)+(-2) \cdot(-3) \\ (-2)\cdot (-4)+2 \cdot (-3) \\ 5\cdot (-4)+3 \cdot (-3)\end{array}\right]$ = $\left[\begin{array}{c}-2 \\ 2 \\ -29\end{array}\right]$ $\approx$ $\left[\begin{array}{c}-1 \\ 7 \\ -26\end{array}\right]$

<font color="red">**Mathematical Background about Linear least squares:**


*The [Linear least squares (LLS)](https://en.wikipedia.org/wiki/Linear_least_squares) is the least squares approximation of linear functions to data. It is a set of formulations for solving statistical problems involved in linear regression. Numerical methods for linear least squares include inverting the matrix of the normal equations and orthogonal decomposition methods.*

*If solutions to a linear system exist but A does not have full column rank, then we have an indeterminate system, all of whose infinitude of solutions are given by this last equation. ([Moore-Penrose Inverse](https://en.m.wikipedia.org/wiki/Moore–Penrose_inverse#Obtaining_all_solutions_of_a_linear_system))*

> Very important articles are [Numerical linear algebra](https://en.m.wikipedia.org/wiki/Numerical_linear_algebra), [Numerical methods for linear least squares](https://en.m.wikipedia.org/wiki/Numerical_methods_for_linear_least_squares), [Eigendecomposition of a matrix](https://en.m.wikipedia.org/wiki/Eigendecomposition_of_a_matrix), and [System of linear equations](https://en.m.wikipedia.org/wiki/System_of_linear_equations), incl. outlook to [Non-Linear Least-Squares](https://en.m.wikipedia.org/wiki/Non-linear_least_squares)

<font color="red">**Mathematical Background about Pseudo-Inverse:**

*The [pseudo-inverse](https://de.m.wikipedia.org/wiki/Pseudoinverse) ist eine Verallgemeinerung der inversen Matrix auf singuläre und nichtquadratische Matrizen, weshalb sie häufig auch als verallgemeinerte Inverse bezeichnet wird. Der häufigste Anwendungsfall für Pseudoinversen ist die Lösung linearer Gleichungssysteme und [linearer Ausgleichsprobleme](https://de.wikipedia.org/wiki/Ausgleichungsrechnung) (curve fitting wie in Regression oder im Machine Learning).*

*The pseudoinverse provides a least squares solution to a system of linear equations.' Siehe auch Linear Least Squares in bezug auf [Moore-Penrose inverse](https://en.m.wikipedia.org/wiki/Moore–Penrose_inverse) (= pseudoinserve = generalization of the inverse matrix).*

* Overdetermined case: A common use of the pseudoinverse is to compute a "best fit" (least squares) solution to a system of linear equations that lacks a solution (see below under § Applications). 

* Underdetermined case: Another use is to find the minimum (Euclidean) norm solution to a system of linear equations with multiple solutions. The pseudoinverse facilitates the statement and proof of results in linear algebra.

* The pseudoinverse is defined and unique for all matrices whose entries are real or complex numbers. It can be computed using the [singular value decomposition](https://en.m.wikipedia.org/wiki/Singular_value_decomposition).

<font color="red">*Underdetermined system*

* *a system of linear equations or a system of polynomial equations is considered underdetermined if there are fewer equations than unknowns*

* *An [underdetermined system of linear equations](https://en.m.wikipedia.org/wiki/Underdetermined_system) has an infinite number of solutions, if any. However, in optimization problems that are subject to linear equality constraints, only one of the solutions is relevant, namely the one giving the highest or lowest value of an objective function.*

  * *In an underdetermined system of linear equations the use of the (Moore Pensore) pseudoinverse is to find the minimum (Euclidean) norm solution to a system of linear equations with multiple solutions.*

  * *The pseudoinverse facilitates the statement and proof of results in linear algebra. The pseudoinverse is defined and unique for all matrices whose entries are real or complex numbers.* 

  * *It can be computed using the singular value decomposition.*

* *The solution set for two equations in three variables is, in general, a line ([Source](https://en.m.wikipedia.org/wiki/System_of_linear_equations#General_behavior)):*

![ggg](https://upload.wikimedia.org/wikipedia/commons/thumb/d/d6/Intersecting_Planes_2.svg/240px-Intersecting_Planes_2.svg.png)

###### <font color="blue">**HHL Approach**

> **Given a matrix $A \in \mathbb{C}^{N \times N}$ and a vector $\vec{b} \in \mathbb{C}^{N}$, find $\vec{x} \in \mathbb{C}^{N}$ satisfying $A \vec{x}=\vec{b}$**

> The spectrum of $A$ is given by: $A\left|v_{j}\right\rangle=\lambda_{j}\left|v_{j}\right\rangle, 1 \geq\left|\lambda_{j}\right| \geq 1 / \kappa$

*One crucial remark to keep in mind is that the classical algorithm returns the full solution, while the HHL can only approximate functions of the solution vector.*

<font color="red">**Solving a system of linear equations with a quantum computer (HHL)**

We want to solve a system of linear equations by finding $\vec{x}$:

> $A \vec{x} = \vec{b}$

Classically you would take the inverse of $A$ (via spectral decomposition / eigendecomposition):

> $\vec{x} = A^{-1} \vec{b}$

The first step towards solving a system of linear equations with a quantum computer is to encode the problem in the quantum language. 

* By rescaling the system, we can assume $\vec{b}$ and $\vec{x}$ to be normalised and map them to the respective quantum states $|b\rangle$ and $|x\rangle$. 

* Usually the mapping used is such that $i^{\text {th }}$ component of $\vec{b}$ (resp. $\vec{x}$ ) corresponds to the amplitude of the $i^{\text {th }}$ basis state of the quantum state $|b\rangle$ (resp. $|x\rangle$ ). 

From now on, we will focus on the rescaled problem

><font color="blue">$A|x\rangle=|b\rangle
$</font> $\quad$ (System of linear equations in a quantum state)

And we want to find this:

><font color="blue">$|x\rangle=A^{-1}|b\rangle$</font> $\quad$ (the solution is: $|x\rangle = \sum_{j=0}^{N-1} \lambda_{j}^{-1} b_{j}\left|u_{j}\right\rangle$)

We need to find the inverse matrix $A^{-1}$. We can get the matrix inverse via eigendecomposition. Since $A$ is Hermitian (normal!), it has a spectral decomposition:
 
>$
A=\sum_{j=0}^{N-1} \lambda_{j}\left|u_{j}\right\rangle\left\langle u_{j}\right|, \quad \lambda_{j} \in \mathbb{R}
$

where $\left|u_{j}\right\rangle$ is the $j^{t h}$ eigenvector of $A$ with respective eigenvalue $\lambda_{j}$. Then,

>$
A^{-1}=\sum_{j=0}^{N-1} \lambda_{j}^{-1}\left|u_{j}\right\rangle\left\langle u_{j}\right|
$

and the right hand side of the system can be written in the eigenbasis of $A$ as

>$
|b\rangle=\sum_{j=0}^{N-1} b_{j}\left|u_{j}\right\rangle, \quad b_{j} \in \mathbb{C}
$

It is useful to keep in mind that the goal of the HHL is to exit the algorithm with the readout register in the state

>$
|x\rangle=A^{-1}|b\rangle=\sum_{j=0}^{N-1} \lambda_{j}^{-1} b_{j}\left|u_{j}\right\rangle
$

Note that here we already have an implicit normalisation constant since we are talking about a quantum state.

https://qiskit.org/textbook/ch-applications/hhl_tutorial.html

*Remember: Hermetian Operator $A$:*

* **Transponierte Matrix**: bijektive, selbstinverse Abbildung einer reellen Matrix

* **Konjugierte Matrizen**: Vertauschen der Vorzeichen

* **Adjungierte Matrix = transponiert + konjugiert** / komplexwertige Matrix, die transponiert + (komplex) conjugiert ist (Vorzeichen umgekehrt)

* **Hermetisch: selbstadjunktiert = symmetrisch** / also wenn man die adjunktierte von A* bildet, kommt wieder A raus: A* = A / Spiegelung an der Diagonalen. Matrix muss quadratisch sein im reellen ist es die Symmetrie

###### <font color="blue">**HHL Algorithm**

**Main Subroutines in HHL:**

* Hamiltonian simulation
* Phase estimation (newer: linear combination of unitaries)
* (Variable-time) amplitude amplification

**HHL-Approach**

1. Prepare the initial state $|b\rangle$. Note that $|b\rangle=\sum_{j} c_{j}\left|v_{j}\right\rangle$.

2. Use the so-called phase estimation algorithm to perform the map
$|b\rangle \rightarrow \sum_{j} c_{j}\left|v_{j}\right\rangle\left|\tilde{\lambda}_{j}\right\rangle$

* $|\tilde{\lambda}_{j}\rangle$ -> This register contains the eigenvalue estimates.

3. Apply a one-qubit conditional rotation to perform the map
$|0\rangle \rightarrow \frac{1}{\kappa \tilde{\lambda}_{j}}|0\rangle+\sqrt{1-\frac{1}{\kappa^{2} \tilde{\lambda}_{j}^{2}}}|1\rangle$

4. Undo step 2 - apply the inverse of phase estimation
$\sum_{j} \frac{c_{j}}{\kappa \tilde{\lambda}_{j}}\left|v_{j}\right\rangle|0\rangle+|\mathrm{bad}\rangle|1\rangle \approx \frac{1}{\kappa A}|b\rangle|0\rangle+|\mathrm{bad}\rangle|1\rangle$

5. Use amplitude amplification to get rid of the „bad“ part of the state with |1>


https://en.m.wikipedia.org/wiki/System_of_linear_equations#Matrix_solution

[Peter Witteg: Quantum Machine Learning - 37 - Overview of the HHL Algorithm](https://m.youtube.com/watch?v=hQpdPM-6wtU)

[Google Quantum: Quantum Algorithms for Systems of Linear Equations (Quantum Summer Symposium 2020)](https://m.youtube.com/watch?v=Xvp56xeNZo4)

https://qiskit.org/textbook/ch-applications/hhl_tutorial.html

In [ ]:
!pip install qiskit

import numpy as np
from qiskit import QuantumCircuit
from qiskit.algorithms.linear_solvers.hhl import HHL
from qiskit.algorithms.linear_solvers.matrices import TridiagonalToeplitz
from qiskit.algorithms.linear_solvers.observables import MatrixFunctional

In [ ]:
matrix = TridiagonalToeplitz(2, 1, 1 / 3, trotter_steps=2)
right_hand_side = [1.0, -2.1, 3.2, -4.3]
observable = MatrixFunctional(1, 1 / 2)
rhs = right_hand_side / np.linalg.norm(right_hand_side)

# Initial state circuit
num_qubits = matrix.num_state_qubits
qc = QuantumCircuit(num_qubits)
qc.isometry(rhs, list(range(num_qubits)), None)

hhl = HHL()
solution = hhl.solve(matrix, qc, observable)
approx_result = solution.observable

https://qiskit.org/documentation/stubs/qiskit.algorithms.HHL.html

https://github.com/quantumlib/Cirq/blob/master/examples/hhl.py